In [2]:
!pip install transformers
!pip install datasets transformers[sentencepiece] sacrebleu
import pandas as pd
import numpy as np
from datasets import load_dataset
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00


In [3]:
# Define dataset path
dataset_path = "cfilt/iitb-english-hindi"
train_dataset = load_dataset(dataset_path, split="train[:1000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [4]:
english_sentences = [item['en'] for item in train_dataset['translation']]
hindi_sentences = [item['hi'] for item in train_dataset['translation']]

In [5]:
from transformers import pipeline

pipe = pipeline("translation", model="facebook/nllb-200-distilled-600M")

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [7]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="hin_Deva", tgt_lang='eng_Latn', max_length = 400)

In [8]:
translatedHinEng = []
for (index, (eng, hin)) in enumerate(zip(english_sentences, hindi_sentences)):
    print(index+1)
    print("Hindi:", hin)
    prediction = translator(hin)
    translation = prediction[0]['translation_text']
    print("Translated: ", translation)
    print("Original: ", eng)
    print("------------------------------")
    translatedHinEng.append(translation)


1
Hindi: अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
Translated:  Give your application the benefit of accessibility exercise
Original:  Give your application an accessibility workout
------------------------------
2
Hindi: एक्सेर्साइसर पहुंचनीयता अन्वेषक
Translated:  Exerciser Accessibility Investigator
Original:  Accerciser Accessibility Explorer
------------------------------
3
Hindi: निचले पटल के लिए डिफोल्ट प्लग-इन खाका
Translated:  Default plug-in template for the lower panel
Original:  The default plugin layout for the bottom panel
------------------------------
4
Hindi: ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
Translated:  Default plug-in template for the upper panel
Original:  The default plugin layout for the top panel
------------------------------
5
Hindi: उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
Translated:  List of plugins that are disabled by default
Original:  A list of plugins that are disabled by default
------------------------------
6
Hindi:

In [9]:
comaparision = pd.DataFrame({'Original': english_sentences, 'Translated': translatedHinEng})
comaparision.to_csv('/content/IITB_Translated_HinEng.csv', index=False)

In [10]:
!pip install nltk

In [11]:
import nltk
from nltk.translate.bleu_score import corpus_bleu


In [12]:
references = [[hindi] for hindi in english_sentences]  # Create reference list of lists
candidates = translatedHinEng  # Model translations


In [13]:
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.5664
